# Experiment 3
All units are in mm

In [1]:
import numpy as np

In [2]:
unit_to_mm = 49.3 / 21.5

In [3]:
yz = [
    (3.1, 7.3),
    (1.3, 4.2),
    (6.7, 3.5),
    (4.5, 1.5)
]

In [12]:
# Uncertainty = 1/2 lowest increment of graduation
uncertainty = 0.5 * unit_to_mm

In [4]:
D = 20

In [14]:
def compute_delta_angle(y, z):
    return np.sqrt(
        uncertainty * 1/2 * (y / np.sqrt(y ** 2 + z ** 2)) / (1 + (y ** 2 + z ** 2) / D ** 2) + 
        uncertainty * 1/2 * (z / np.sqrt(y ** 2 + z ** 2)) / (1 + (y ** 2 + z ** 2) / D ** 2)
    )

In [17]:
angles = []

for point, (y, z) in zip(['A', 'B', 'C', 'D'], yz):
    deg = np.degrees(1/2 * np.arcsin(np.sqrt(y ** 2 + z ** 2) * unit_to_mm / D))
    delta_deg = compute_delta_angle(y, z)
    print(f"{deg:.2f}+-{delta_deg:.2f}")
    angles.append(
        (point, deg, delta_deg)
    )

32.70+-0.81
15.13+-0.83
30.04+-0.82
16.47+-0.83


In [6]:
degree_expected = []
for h in range(4):
    for k in range(4):
        for l in range(4):
            degree_expected.append(
                (
                    f"{h, k, l}", np.degrees(np.sin(h / np.sqrt(h ** 2 + k ** 2 + l ** 2))) 
                )
            )

/tmp/ipykernel_53560/3315925370.py:7: RuntimeWarning: invalid value encountered in double_scalars
  f"{h, k, l}", np.degrees(np.sin(h / np.sqrt(h ** 2 + k ** 2 + l ** 2)))


In [20]:
def find_similar_planes(degree, delta_degree):
    similar = []
    for plane, theoretical_degree in degree_expected:
        # This should theoretical be best if we could find the uncertainty but not sure if there is an easy way of doing this :(
        if abs(degree - theoretical_degree) < delta_degree:
            similar.append((plane, theoretical_degree))
    return similar

In [21]:
for point, deg, delta in angles:
    print(point)
    print(find_similar_planes(deg, delta))

A
[]
B
[('(1, 2, 3)', 15.131294224366904), ('(1, 3, 2)', 15.131294224366904)]
C
[('(2, 0, 3)', 30.177026630572893), ('(2, 3, 0)', 30.177026630572893)]
D
[('(1, 1, 3)', 17.014767321817022), ('(1, 3, 1)', 17.014767321817022)]
